In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = 'data/mnist/'

In [3]:
raw_train = pd.read_csv(path + 'train.csv')
raw_test = pd.read_csv(path + 'train.csv')

In [4]:
raw_train_array = raw_train.values
raw_test_array = raw_test.values

In [5]:
raw_train_array = np.random.permutation(raw_train_array)
len(raw_train_array)

42000

In [6]:
raw_train = raw_train_array[:40000, :]
raw_valid = raw_train_array[40000:, :]

In [7]:
# train_label = np.eye(10)[raw_train[:,0]]
train_label = raw_train[:,0]
train_data = raw_train[:,1:]
# valid_label = np.eye(10)[raw_valid[:,0]]
valid_label = raw_valid[:,0]
valid_data = raw_valid[:,1:]

train_data.shape

(40000, 784)

In [8]:
def reshape(data, target_size): return np.reshape(data, target_size)

In [9]:
train_data = reshape(train_data, [40000, 1, 28, 28])
valid_data = reshape(valid_data, [2000, 1, 28, 28])
train_data.shape, train_label.shape, valid_label.shape, valid_data.shape

((40000, 1, 28, 28), (40000,), (2000,), (2000, 1, 28, 28))

In [10]:
BATCH_SIZE = 64
LEARNING_RATE = 0.1
EPOCH = 2

In [11]:
#convert to pytorch tensor
train_data = torch.from_numpy(train_data)..type(torch.FloatTensor)
train_label = torch.from_numpy(train_label).type(torch.LongTensor)
val_data = torch.from_numpy(valid_data).type(torch.FloatTensor)
val_label = torch.from_numpy(valid_label).type(torch.LongTensor)

train_data.size(),train_label.size(),val_data.size(),val_label.size()

(torch.Size([40000, 1, 28, 28]),
 torch.Size([40000]),
 torch.Size([2000, 1, 28, 28]),
 torch.Size([2000]))

In [12]:
train_dataset = Data.TensorDataset(data_tensor=train_data, target_tensor=train_label)
val_dataset = Data.TensorDataset(data_tensor=val_data, target_tensor=val_label)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [13]:
#pyton opp
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #in_chanel out_chanel kernel stride padding
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 10)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = F.relu(self.conv3(x))        
        x = F.max_pool2d(F.relu(self.conv4(x)), 2) 
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
             
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
cnn = CNN()
print(cnn)

CNN (
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear (1024 -> 512)
  (fc2): Linear (512 -> 10)
)


In [14]:
list(cnn.parameters())[2].size() #conv2 weights

torch.Size([32, 32, 3, 3])

In [15]:
#Loss and Optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LEARNING_RATE)

In [17]:
#train the model
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
#         print(type(images))
#         print(type(labels))
        
        images = Variable(images)
        labels = Variable(labels)
        #print(type(images))
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(loss.data)
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, 2, i+1, len(train_dataset)//BATCH_SIZE, loss.data[0]))
        

RuntimeError: expected Long tensor (got Float tensor)

In [ ]:
# save and load(保存和提取)

In [63]:
# save
def save():
    pass
    #torch.save(net_name, 'net.pkl')
    #torch.save(net_name.state_dict(), 'net_params.pkl')
# load
def restore_net():
    pass
    #net_new = torch.load('net.pkl')

def restore_params():
    pass
    #net_new_old_params = NET()
    #net_new_old_params = net_new_old_params.load_state_dict(torch.load()'net_params.pkl'))


In [65]:
#批处理

In [72]:
#optimizer 优化器
# optimizer = torch.optim.SGD()
# torch.optim.Adam
# momentum (m)
# alpha (RMSprop)
# Adam (betas)